链式法则与反向传播算法
===

# 1.链式法则
若函数$\mu=\phi(t),\nu=\psi(t)$，在点t可到，$z=f(\mu,\nu)$，那么就有
$$\frac{\partial{z}}{\partial{t}}=\frac{\partial{z}}{\partial{\mu}} \bullet \frac{\partial{\mu}}{\partial{t}} + \frac{\partial{z}}{\partial{\nu}} \bullet \frac{\partial{\nu}}{\partial{t}}$$
其实就是
$$\frac{\partial{f}}{\partial{\omega}}=\frac{\partial{f}}{\partial{q}} \bullet \frac{\partial{q}}{\partial{\omega}}$$

# 2.反向传播算法
神经网络是一个模型，权值就是模型的参数，也就是模型需要学习的东西。而神经网络的连结方式、网络的层次、每层的节点数这些参数是人为事先设置的，这些参数叫做超参数。假设每个训练样本为$(\overrightarrow{x}, \overrightarrow{t})$，其中向量$\overrightarrow{x}$是训练样本的特征，而
$\overrightarrow{t}$是样本的目标值<br/>
![images](Images/03/00/04_001.png)

# 3.实例一
函数$f(x,y,z)=(x+y)*z$，且x=-2,y=5,z=-4使用链式法则求出$\frac{\partial{f}}{\partial{x}}\frac{\partial{f}}{\partial{y}}\frac{\partial{f}}{\partial{z}}$<br/>
解:令$q=x+y,\therefore f=q*z$

前向计算:$q=x+y=-2+5=3,f=q*z=3*-4=-12$

反向传播:

$$\begin{split}
\frac{\partial{f}}{\partial{x}}&=\frac{\partial{f}}{\partial{q}} \bullet \frac{\partial{q}}{\partial{x}}=z \bullet 1=-4 \\
\frac{\partial{f}}{\partial{y}}&=\frac{\partial{f}}{\partial{q}} \bullet \frac{\partial{q}}{\partial{y}}=z \bullet 1=-4 \\
\frac{\partial{f}}{\partial{z}}&=q=3
\end{split}$$
![images](Images/03/00/04_002.png)<br/>
上图中，反向传播的起点的1是因为要求的是$\frac{\partial{f}}{\partial{f}}$

# 4.实例二
![images](Images/03/00/04_003.png)<br/>

# 5.实例三
![images](Images/03/00/04_004.png)

# 6.实例四
$$Sigmoid=f(\omega,x)=\frac{1}{1+e^{-(\omega_0x_0+\omega_1x_1+\omega_2)}}$$
前向计算很简单，至于反向传播，可以把sigmoid函数拆分成如下小函数
$$\begin{eqnarray}
f(x)=\frac{1}{x} &\Rightarrow& \frac{\partial{f}}{\partial{x}}=-\frac{1}{x^2} \\
f_c(x)=c+x &\Rightarrow& \frac{\partial{f}}{\partial{x}}=1 \\
f(x)=e^x &\Rightarrow& \frac{\partial{f}}{\partial{x}}=e^x\\
f_a(x)=ax &\Rightarrow& \frac{\partial{f}}{\partial{x}}=a
\end{eqnarray}$$
![images](Images/03/00/04_005.png)<br/>
-0.53是怎么来的，在前向传播中，q=1.37，然后求出$\frac{f}{q}=\frac{1}{1.37}=0.73$，这个就是最终的f的值，那么反向传播中，第一步就是求出$\frac{\partial{f}}{\partial{q}}=-\frac{1}{q^2}=-\frac{1}{1.37*1.37}=-0.53$

# 7.具体的做法
![image](Images/03/00/04_006.jpg)
可以看到输入的数据$x$,经过一个函数操作$F.forward$得到下一层的输入数据$x_1$，循环如此，整个数据向前流动。$F.f$函数可以包含参数，例如上面公式里写的$W_1,b_1$，对于这些所有这些参数我们统称为$\theta$.

通过这个流动的过程，我们已经可以得到$Loss=J(\theta,x,y)$，由于我们的目标是要用梯度去求解一个优化问题，所以必须求出
$$\frac{\partial{J(\theta,x,y)}}{\partial{\theta}}$$
![image](Images/03/00/04_007.jpg)
这样看来$F.forward$只需要完成以下两个功能即可使得整个导数的信息流动起来,从而完成整个网络参数的求导：
- 根据传入的$\frac{\partial{loss}}{\partial{x_1}}$求解出$\frac{\partial{loss}}{\partial{x}}$，保证导数信息可以继续向前传播
- 定义与$F.forward$操作所对应的$\frac{\partial{x_1}}{\partial{W_1}}$,$\frac{\partial{x_1}}{\partial{b_1}}$, 结合传入的$\frac{\partial{loss}}{\partial{x_1}}$，求出$w_1$,$b_1$的导数 ($\frac{\partial{x_1}}{\partial{W_1}}$就是$\frac{\partial{F.forward}}{\partial{W_1}}$)

注意这里的$F$只是一个代指，上图里对应的是多层感知机的某一层。实际上它也可以是卷积层，池化层，等等(多输入操作符需要注意一下)，也就是说实现网络中的一个操作的反向传播，基本操作只需要完成以上这两点，1是保证导数信息(残差)继续在网络中向前传播. 2是根据传入的导数信息求解自身参数的导数。如果没有参数例如大多数activation,这个部分就可以省略。

我们知道有参数的层，前向计算都是按照公式
$$\vec{Y}=\vec{x} \odot \vec{W} + \vec{b}$$
在反向传播算法中，我们已知的是上一层传播过来的梯度，也就是$\frac{\partial{Loss}}{\partial{\vec{Y}}}$，目的是要求出$\frac{\partial{L}}{\partial{\vec{X}}},\frac{\partial{L}}{\partial{\vec{W}}},\frac{\partial{L}}{\partial{\vec{b}}}$。于是有下列公式
- $\frac{\partial{L}}{\partial{\vec{X}}}=\frac{\partial{L}}{\partial{\vec{Y}}} \odot \vec{W}^T$
- $\frac{\partial{L}}{\partial{\vec{W}}}=\vec{X}^T \odot \frac{\partial{L}}{\partial{\vec{Y}}}$
- $\frac{\partial{L}}{\partial{\vec{b}}}=\vec{1}^T \odot \frac{\partial{L}}{\partial{\vec{Y}}}$

其中$\vec{X}$是$n \times p$, $\vec{W}$是$p \times m$，$\vec{Y}$是$n \times m$

## 7.1.公式一推导
记$L$关于Y的函数为$L=L(y_{11},y_{12},...,y_{nm})$,$y_{st}$关于$X$的函数记为$y_{st}=f_{st}(x_{11},x_{12},...,x_{np})$,则根据链式法则，有
$$\frac{\partial{L}}{\partial{x_{ij}}}=\sum_s\sum_t\frac{\partial{L}}{\partial{y_{st}}} \times \frac{\partial{y_{st}}}{\partial{x_{ij}}}$$
对于$\frac{\partial{y_{st}}}{\partial{x_{ij}}}$,我们知道只有$s=i$的情况下才有梯度，所以，上式就能简化为
$$\begin{split}
\frac{\partial{L}}{\partial{x_{ij}}}&=sum_t\frac{\partial{L}}{\partial{y_{it}}} \times \frac{\partial{y_{it}}}{\partial{x_{ij}}} \\
&=sum_t\frac{\partial{L}}{\partial{y_{it}}} \times w_{jt}
\end{split}$$
由于$\frac{\partial{L}}{\partial{y_{it}}}$是$\frac{\partial{L}}{\partial{Y}}$的第i行，$w_{jt}$是$W^T$的第j列，上述公式对所有(i,j)都成立，因此公式得证。

## 7.2.公式二推导

# 8.总结

## 8.1.前向计算
神经网络的前向计算方法就是做如下的循环操作
$$Y_1=W_1 \bullet X_1+b_1,X_2=f_1(Y_1);Y_2=W_2 \bullet X_2+b_2,X_3=f_2(Y_2);...;Loss=C(X_L)$$
其中C为损失函数，L为神经网络最后一层

## 8.2.反向传播算法

### 8.2.1.最后一层
首先计算$\frac{\partial{C}}{\partial{X_L}}$，不同的损失函数导数是不一样的。由于损失函数没有参数，所以一般情况下这一层只需要计算这个梯度。  

### 8.2.2.倒数第二层
- $\frac{\partial{C}}{\partial{Y_{L-1}}}=f_{L-1}'(X_{L-1}) \times \frac{\partial{C}}{\partial{X_L}}$
- $\frac{\partial{C}}{\partial{W_{L-1}}}=\frac{\partial{C}}{\partial{Y_{L-1}}} \bullet X_{L-1}^T$
- $\frac{\partial{C}}{\partial{X_{L-1}}}=W_{L-1}^T \bullet \frac{\partial{C}}{\partial{Y_{L-1}}}$
- $\frac{\partial{C}}{\partial{B_{L-1}}}=\frac{\partial{C}}{\partial{Y_{L-1}}}$

### 8.2.3.倒数第三层
- $\frac{\partial{C}}{\partial{Y_{L-2}}}=f_{L-2}'(X_{L-2}) \times \frac{\partial{C}}{\partial{X_{L-1}}}$
- $\frac{\partial{C}}{\partial{W_{L-2}}}=\frac{\partial{C}}{\partial{Y_{L-2}}} \bullet X_{L-2}^T$
- $\frac{\partial{C}}{\partial{X_{L-2}}}=W_{L-2}^T \bullet \frac{\partial{C}}{\partial{Y_{L-2}}}$
- $\frac{\partial{C}}{\partial{B_{L-2}}}=\frac{\partial{C}}{\partial{Y_{L-2}}}$

以此类推

## 8.3.举例
$Y,W,X,B$的所有元素都是代价函数C的变量。已知$Y_{2 \times 1}=W_{2 \times 3} \bullet X_{3 \times 1}+B_{2 \times 1}$，即
$$\begin{cases}
y_1&=w_{11}x_1+w_{12}x_2+w_{13}x_3+b_1 \\
y_2&=w_{21}x_1+w_{22}x_2+w_{23}x_3+b_2
\end{cases}$$
$\frac{\partial{C}}{\partial{Y}}=\begin{bmatrix}
\frac{\partial{C}}{\partial{y_1}} \\
\frac{\partial{C}}{\partial{y_2}}
\end{bmatrix}$，求$\frac{\partial{C}}{\partial{W}},\frac{\partial{C}}{\partial{X}},\frac{\partial{C}}{\partial{B}}$.

我们根据上面的式子，可以知道如下关系
$$\begin{split}
\frac{\partial{y_i}}{\partial{w_{ij}}}&=x_j \\
\frac{\partial{y_i}}{\partial{x_j}}&=w_{ij} \\
\frac{\partial{y_i}}{\partial{b_i}}&=1
\end{split}$$
从而，有
$$\frac{\partial{C}}{\partial{w_{ij}}}=\frac{\partial{C}}{\partial{y_i}}\frac{\partial{y_i}}{\partial{w_{ij}}}=x_j\frac{\partial{C}}{\partial{y_i}}$$
因此就有
$$\begin{split}
\frac{\partial{C}}{\partial{W}}&=\begin{bmatrix}
x_1\frac{\partial{C}}{\partial{y_1}}&x_2\frac{\partial{C}}{\partial{y_1}}&x_3\frac{\partial{C}}{\partial{y_1}} \\
x_1\frac{\partial{C}}{\partial{y_2}}&x_2\frac{\partial{C}}{\partial{y_2}}&x_3\frac{\partial{C}}{\partial{y_2}}
\end{bmatrix} \\
&=\begin{bmatrix}
\frac{\partial{C}}{\partial{y_1}} \\
\frac{\partial{C}}{\partial{y_2}}
\end{bmatrix} \bullet \begin{bmatrix}
x_1&x_2&x_3
\end{bmatrix} \\
&=\frac{\partial{C}}{\partial{Y}} \bullet X^T
\end{split}$$
下面求$\frac{\partial{C}}{\partial{X}}$
$$\begin{split}
\because \frac{\partial{C}}{\partial{x_j}}&=\frac{\partial{C}}{\partial{y_1}}\frac{\partial{y_1}}{\partial{x_j}}+\frac{\partial{C}}{\partial{y_2}}\frac{\partial{y_2}}{\partial{x_j}} \\
\therefore \frac{\partial{C}}{\partial{X}}&=\begin{bmatrix}
\frac{\partial{C}}{\partial{x_1}} \\
\frac{\partial{C}}{\partial{x_2}} \\
\frac{\partial{C}}{\partial{x_3}}
\end{bmatrix} \\
&=\begin{bmatrix}
\frac{\partial{C}}{\partial{y_1}}\frac{\partial{y_1}}{\partial{x_1}}+\frac{\partial{C}}{\partial{y_2}}\frac{\partial{y_2}}{\partial{x_1}} \\
\frac{\partial{C}}{\partial{y_1}}\frac{\partial{y_1}}{\partial{x_2}}+\frac{\partial{C}}{\partial{y_2}}\frac{\partial{y_2}}{\partial{x_2}} \\
\frac{\partial{C}}{\partial{y_1}}\frac{\partial{y_1}}{\partial{x_3}}+\frac{\partial{C}}{\partial{y_2}}\frac{\partial{y_2}}{\partial{x_3}}
\end{bmatrix} \\
&=\begin{bmatrix}
w_{11}\frac{\partial{y_1}}{\partial{x_1}}+w_{21}\frac{\partial{y_2}}{\partial{x_1}} \\
w_{12}\frac{\partial{y_1}}{\partial{x_2}}+w_{22}\frac{\partial{y_2}}{\partial{x_2}} \\
w_{13}\frac{\partial{y_1}}{\partial{x_3}}+w_{23}\frac{\partial{y_2}}{\partial{x_3}}
\end{bmatrix} \\
&=\begin{bmatrix}
w_{11}&w_{21} \\
w_{12}&w_{22} \\
w_{13}&w_{23}
\end{bmatrix} \bullet \begin{bmatrix}
\frac{\partial{C}}{\partial{y_1}} \\
\frac{\partial{C}}{\partial{y_2}}
\end{bmatrix} \\
&=W^T \bullet \frac{\partial{C}}{\partial{Y}}
\end{split}$$

## 8.4.推广
- 如果$Y_{p \times r}=W_{p \times q} \bullet X_{q \times r}$，那么$(\frac{\partial{C}}{\partial{W}})_{p \times q}=(\frac{\partial{C}}{\partial{W}})_{p \times q} \bullet (X^T)_{r \times q}$
- 如果$Y_{p \times r}=W_{p \times q} \bullet X_{q \times r}$，那么$(\frac{\partial{C}}{\partial{X}})_{q \times r}=(W^T)_{q \times p} \bullet (\frac{\partial{C}}{\partial{Y}})_{p \times r}$